In [14]:
import geopandas as gpd
import numpy as np
import shutil
import sys

sys.path.append("/home/nilscp/GIT/crater_morphometry")
import geomorphometry
from preprocessing import DEM_extraction
from rim_detection import rim_detection
from pathlib import Path

## Clip DEM and visible image for rayed craters

In [8]:
location_of_craters = "/home/nilscp/tmp/fresh_impact_craters/GLD100/crater_center/GLD100_rayed_craters_larger_than_1500m.shp"
dem = "/home/nilscp/QGIS/Moon/globalMosaics/Lunar_LRO_WAC_GLD100_DTM_79S79N_100m_v1.1.tif"
clip_distance = 8.0
output_dir = "/home/nilscp/tmp/fresh_impact_craters/GLD100/GLD100/"
identifier_dem = "GLD100DEM"
shp_folder = "/home/nilscp/tmp/fresh_impact_craters/GLD100/shapefiles/"

In [5]:
DEM_extraction.clip_raster_to_crater(location_of_craters, dem, clip_distance, output_dir, shp_folder, identifier_dem, craterID = None)

100%|██████████| 852/852 [19:44<00:00,  1.39s/it]


## Generate regional (2.0-3.0R) detrended DEM (SLDEM2015, Rayed craters)

In [9]:
scaling_factor = 1.0
dem_folder = "/home/nilscp/tmp/fresh_impact_craters/GLD100/GLD100/"
dem_detrended_folder = "/home/nilscp/tmp/fresh_impact_craters/GLD100/GLD100_detrended/"
craterID = None
overwrite = True

In [10]:
rim_detection.generated_detrended_dem(location_of_craters, scaling_factor, dem_folder, shp_folder, dem_detrended_folder, craterID=craterID, overwrite=overwrite)

  7%|▋         | 59/852 [00:27<05:34,  2.37it/s]/home/nilscp/.pyenv/versions/3.8.0/envs/computer-vision/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/nilscp/.pyenv/versions/3.8.0/envs/computer-vision/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
100%|██████████| 852/852 [06:12<00:00,  2.29it/s]


## The detection of the rim composite, and the following update of the crater centre
----
#### Note that the algorithm will not work if:
- the centre of the crater is outside of the crater of interest
- the resolution of the dem is too low (at least 20 cells is required along the crater radius to provide reasonable results <br>
----
#### It can be that the algorithm work but gives wrong results if:
- There are large change in slopes that does not correspond to the actual change in slope associated with the rim (either very close or very far)

In [11]:
suffix = '_GLD100DEM_detrended.tif'

In [12]:
rim_detection.main(location_of_craters, dem_detrended_folder, shp_folder, suffix, threshold_min=None, threshold_max=None , craterID = None)

  0%|          | 0/849 [00:00<?, ?it/s]/home/nilscp/GIT/crater_morphometry/rim_detection/rim_detection.py:1033: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(filename)
 11%|█         | 91/849 [07:16<50:58,  4.04s/it]  

crater0126


100%|██████████| 849/849 [1:03:02<00:00,  4.46s/it]


## Manual check of results
It is important to manually check if the detected rims give meaningful results. If not, the polygon can be deleted, and replace by a manually generated ellipse. This should (hopefully) not happen in many cases for fresh impact structures (<5%). However, this is likely to happen more often for degraded craters (such as the rayed crater dataset, that contains both young and old craters). For completion, we are writing down the id of the crater where a problem is detected, to understand if we can improve the rule-based detection of rim in the future. 

## Update centre of craters
- Merge the centres of craters together
- Three craters were deleted because they were closed to the origin (and that they overlapped between the edge of the global raster)
- This is not going to be any issues because of that, as I only want to computer GLD100 to have an idea of how SLDEM2015 is performing compare to it.

In this special case, I am just copying the ellipse of the SLDEM2015. It should be a minimal shift between the two datasets, and the computation with GLD100 DTM is only for comparison and not use in the final results.

## let's copy the ellipse and change their names

In [37]:
input_folder = Path("/home/nilscp/tmp/fresh_impact_craters/shapefiles/polygons")
desti_folder = Path("/home/nilscp/tmp/fresh_impact_craters/GLD100/shapefiles/polygons")
input_txt_str = "_LROKaguyaDEM_detrended_ellipse_candidate2_polygon"
desti_txt_str = "_GLD100DEM_detrended_ellipse_candidate2_polygon"
gdf = gpd.read_file(location_of_craters)    

In [38]:
for i, row in gdf.iterrows():
    tmp_files = list(input_folder.glob(row.CRATER_ID + input_txt_str + "*"))
    for file in tmp_files:
        ndesti = file.name.replace(input_txt_str, desti_txt_str)
        ndesti = desti_folder / ndesti
        shutil.copy(file, ndesti)

## Calculate morphometrical parameters from the second ellipse, final crater centre and new detrended products
- load ellipse in world coordinates (can also give the possibility to use points and not the ellipse!)
- create an ellipse with 2.0 r (maybe need to modify with the -xc, yc

There are a few problems by using ellipse... approximating the rim of a crater with an ellipse is totally ok, but it will always result in "shooting" over or under the actual position of the rim of crater. The majority of morphometric parameters (e.g., middle, upper, rim flank slope) are a bit sensitive to the exact location of the rim. I think I will try to create a function that retrieve the breaks in slope within 10% of the ellipse, and then use those few locations to extract cross sections and calculate parameters. This means that not all of the craters will have a constant number of cross-sections (512) as for ellipses. I will compare the results of those different values.

In [ ]:
location_of_craters = "/home/nilscp/tmp/fresh_impact_craters/GLD100/crater_center/GLD100_rayed_craters_larger_than_1500m.shp"
shp_folder = '/home/nilscp/tmp/fresh_impact_craters/GLD100/shapefiles/polygons/'
dem_folder = '/home/nilscp/tmp/fresh_impact_craters/GLD100/GLD100_detrended/'
suffix = '_GLD100DEM_detrended.tif'
prefix = 'crater'
craterID=None